# 🔥 Qwen2.5-VL Model Attribution (Vision)

In [1]:
import os

# Set the huggingface mirror and cache path
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com" # for Chinese
os.environ["HF_HOME"] = "../model_checkpoint/hf_cache"


from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

import torch

/home/cry/anaconda3/envs/qwen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Load Qwen2.5-VL Model

In [2]:
# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)
model.eval()

# default processor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.06it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "../demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)

### 2. Data Proccessing

In [4]:
inputs = processor(
    text=[text],
    images=image_inputs,    # 这里可以多个
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)    # dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])

### 3. Inference

In [5]:
# Inference: Generation of the output
with torch.no_grad():
    generated_ids = model.generate(
        **inputs, 
        do_sample=False,      # 禁用采样，改用贪婪搜索
        num_beams=1,          # 设置为1，确保是贪婪而不是beam search
        max_new_tokens=128)
    generated_ids_trimmed = [   # 去掉图像和prompt的文本
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

/home/cry/anaconda3/envs/qwen/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-06` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [6]:
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['The image depicts a serene beach scene with a person and a dog. The person is sitting on the sandy beach, facing the ocean, and appears to be interacting with the dog. The dog is also sitting on the sand, facing the person, and seems to be giving a paw or a friendly gesture. The person is wearing a plaid shirt and has long hair. The background shows the ocean with gentle waves and a clear sky, suggesting it might be early morning or late afternoon due to the soft lighting. The overall atmosphere of the image is calm and joyful, capturing a moment of connection between the person and their dog at the beach.']


In [8]:
tokenizer = processor.tokenizer
tokens = tokenizer.convert_ids_to_tokens(generated_ids_trimmed[0])
print(tokens[:14])

['The', 'Ġimage', 'Ġdepicts', 'Ġa', 'Ġserene', 'Ġbeach', 'Ġscene', 'Ġwith', 'Ġa', 'Ġperson', 'Ġand', 'Ġa', 'Ġdog', '.']


In [9]:
generated_ids_trimmed[0][:14]

tensor([  785,  2168, 61891,   264, 94763, 11321,  6109,   448,   264,  1697,
          323,   264,  5562,    13], device='cuda:0')

In [ ]:
tokens_org = tokenizer.convert_ids_to_tokens(generated_ids[0])
print(tokens_org[3602:3602+14])

['The', 'Ġimage', 'Ġdepicts', 'Ġa', 'Ġserene', 'Ġbeach', 'Ġscene', 'Ġwith', 'Ġa', 'Ġperson', 'Ġand', 'Ġa', 'Ġdog']


In [28]:
generated_ids[0][3602:3602+13]

tensor([  785,  2168, 61891,   264, 94763, 11321,  6109,   448,   264,  1697,
          323,   264,  5562], device='cuda:0')

In [19]:
len(inputs['input_ids'][0])

3602

In [7]:
generated_ids_trimmed[0]

tensor([  785,  2168, 61891,   264, 94763, 11321,  6109,   448,   264,  1697,
          323,   264,  5562,    13,   576,  1697,   374, 11699,   389,   279,
        67439, 11321,    11, 12880,   279, 17951,    11,   323,  7952,   311,
          387, 44730,   448,   279,  5562,    13,   576,  5562,   374,  1083,
        11699,   389,   279,  9278,    11, 12880,   279,  1697,    11,   323,
         4977,   311,   387,  7086,   264, 76838,   476,   264, 11657, 30157,
           13,   576,  1697,   374, 12233,   264,   625,  3779, 15478,   323,
          702,  1293,  6869,    13,   576,  4004,  4933,   279, 17951,   448,
        21700, 16876,   323,   264,  2797, 12884,    11, 22561,   432,  2578,
          387,  4124,  6556,   476,  3309, 13354,  4152,   311,   279,  8413,
        17716,    13,   576,  8084, 16566,   315,   279,  2168,   374, 19300,
          323, 82112,    11, 39780,   264,  4445,   315,  3633,  1948,   279,
         1697,   323,   862,  5562,   518,   279, 11321,    13],

### 4. Get the target token probability

In [10]:
inputs_new = inputs.copy()
inputs_new['input_ids'] = generated_ids
inputs_new['attention_mask'] = torch.ones_like(generated_ids)
inputs_new = inputs_new.to(model.device)

In [11]:
# Forward calculation to get all logits (including the logits of the input part)
with torch.no_grad():
    outputs = model(
        **inputs_new,
        return_dict=True,
        use_cache=True,
    )
    all_logits = outputs.logits  # [batch_size, seq_len, vocab_size]

In [13]:
all_logits[0,3601].argmax()

tensor(785, device='cuda:0')

In [14]:
generated_ids[0,3602]

tensor(785, device='cuda:0')

In [29]:
generated_ids[0,8]

tensor(13, device='cuda:0')